In [172]:
import torch
import numpy as np
import random
import torch.nn as nn
from torch.optim import SGD
import torch.nn.functional as F

In [173]:
from torch_geometric.datasets import KarateClub

dataset = KarateClub()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')


data = dataset[0]  # Get the first graph object.

print(data)
print('==============================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Dataset: KarateClub():
Number of graphs: 1
Number of features: 34
Number of classes: 4
Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])
Number of nodes: 34
Number of edges: 156
Average node degree: 4.59
Number of training nodes: 4
Training node label rate: 0.12
Has isolated nodes: False
Has self-loops: False
Is undirected: True


![](https://i.imgur.com/NUJjZ1q.png)

In [174]:
adj_dense = torch.sparse_coo_tensor(indices=data.edge_index, values=torch.ones(data.num_edges)).to_dense()

# Transform Adjacency Matrix to Adjacency List
neigs = {}
########  TODO  ########
for idx, node in enumerate(adj_dense):
    neigs[idx] = torch.where(node == 1)[0].tolist()
########################
neigs

{0: [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 17, 19, 21, 31],
 1: [0, 2, 3, 7, 13, 17, 19, 21, 30],
 2: [0, 1, 3, 7, 8, 9, 13, 27, 28, 32],
 3: [0, 1, 2, 7, 12, 13],
 4: [0, 6, 10],
 5: [0, 6, 10, 16],
 6: [0, 4, 5, 16],
 7: [0, 1, 2, 3],
 8: [0, 2, 30, 32, 33],
 9: [2, 33],
 10: [0, 4, 5],
 11: [0],
 12: [0, 3],
 13: [0, 1, 2, 3, 33],
 14: [32, 33],
 15: [32, 33],
 16: [5, 6],
 17: [0, 1],
 18: [32, 33],
 19: [0, 1, 33],
 20: [32, 33],
 21: [0, 1],
 22: [32, 33],
 23: [25, 27, 29, 32, 33],
 24: [25, 27, 31],
 25: [23, 24, 31],
 26: [29, 33],
 27: [2, 23, 24, 33],
 28: [2, 31, 33],
 29: [23, 26, 32, 33],
 30: [1, 8, 32, 33],
 31: [0, 24, 25, 28, 32, 33],
 32: [2, 8, 14, 15, 18, 20, 22, 23, 29, 30, 31, 33],
 33: [8, 9, 13, 14, 15, 18, 19, 20, 22, 23, 26, 27, 28, 29, 30, 31, 32]}

## Implement RandomWalk Function

In [175]:
def random_walk(adj_list, start_node, num_length, p ,q):
    """
    adj_list [dict] : graph structure
    start_node [int] :
    num_length [int] : length of randomwalk
    p [int, float] : weight of BFS
    q [int, float] : weight of DFS
    """
    walk = [start_node]
    
    while len(walk) < num_length:
        cur_node = walk[-1]
        neigs = adj_list[cur_node]

        if len(walk) == 1:
            next_node = random.choice(neigs)
        else:
            prev_node = walk[-2]
            prob = []
            for neig in neigs:
                ########  TODO  ########
                # BFS
                if neig == prev_node:    
                    prob.append(1/p)
                
                # same dist. to start
                elif neigs in adj_list[prev_node]: 
                    prob.append(1)
                
                # DFS
                else : 
                    prob.append(1/q)
                ########################

            norm_prob = (np.array(prob) / np.array(prob).sum())
            next_node = random.choices(neigs, norm_prob.tolist())[0]

        walk.append(next_node)
    return walk

## Constructure Training data

In [176]:
def generate_training_data(graph, num_walks, walk_length, context_size, p=4, q=1):
    """
    You should constructure positive datset and negative dataset.
    Shape of dataset : (Numbers of node * num_walks, walk_length)
    """
    pos_walks = []
    neg_walks = []
    for _ in range(num_walks):
        for node in graph.keys():
            pos_path = random_walk(graph, node, walk_length, p, q)
            ########  TODO  ########
            #Implement Negative Sampling
            neg_path = []
            

            ########################
            pos_walks.append(pos_path)
            neg_walks.append(neg_path)
    # print(pos_walks)
    pos_walks = np.vstack(pos_walks)
    neg_walks = np.vstack(neg_walks)

    pos_dataset = []
    neg_dataset = []
    num_walks_per_rw = walk_length + 1 - context_size
    for i in range(num_walks_per_rw):
        pos_data = pos_walks[:, i:i + context_size]
        neg_data = neg_walks[:, i:i + context_size]
        pos_dataset.append(pos_data)
        neg_dataset.append(neg_data)
    pos_dataset = np.vstack(pos_dataset)
    neg_dataset = np.vstack(neg_dataset)

    return np.array(pos_dataset), np.array(neg_dataset)


**Node2Vec loss fuction**

\begin{equation}
L(\Theta) = \log \left ( \sigma (z_u^{\top} z_v)  \right) - \sum_{i=1}^k \log \left ( \sigma (z_u^{\top} z_{n_i})  \right), n_i \sim P_V
\end{equation}

In [177]:
class Node2Vec(nn.Module):
    def __init__(self, num_nodes, embed_dim):
        super(Node2Vec, self).__init__()
        self.num_nodes = num_nodes
        self.embed_dim = embed_dim
        self.EPS = 1e-5
        self.embeddings = nn.Embedding(num_nodes, embed_dim)
        
    def forward(self, pos_set, neg_set):
        # calculate the loss of positive pairs
        start_nodes, pos_nodes = pos_set[:, 0], pos_set[:, 1:]

        start_nodes = torch.tensor(start_nodes, dtype=torch.long)
        pos_nodes = torch.tensor(pos_nodes, dtype=torch.long)
        
        ########  TODO  ########


        pos_loss = None
        ########################

        # calculate the loss of negative pairs
        start_nodes, pos_nodes = neg_set[:, 0], neg_set[:, 1:]

        start_nodes = torch.tensor(start_nodes, dtype=torch.long)
        pos_nodes = torch.tensor(pos_nodes, dtype=torch.long)
        
        ########  TODO  ########


        neg_loss = None
        ########################

        return pos_loss + neg_loss


In [178]:
# 設定參數
walks_per_node = 10
walk_length = 10
context_size = 10
embedding_dim = 16
epochs = 100
learning_rate = 0.01
p=4
q=1

# 生成訓練數據
train_set = generate_training_data(neigs, walks_per_node, walk_length, context_size, p, q)
model = Node2Vec(num_nodes=data.num_nodes, embed_dim=embedding_dim)
optimizer = SGD(model.parameters(), lr=learning_rate)
pos_set, neg_set = train_set

print(pos_set.shape)

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])
(340, 10)


In [179]:
def train_model():
    model.train()
    pos_set, neg_set = train_set
    # 計算損失
    loss = model(pos_set, neg_set)
    # 反向傳播和更新
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss

In [180]:
for epoch in range(1, 201):
    loss = train_model()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

IndexError: index 0 is out of bounds for axis 1 with size 0